In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
import lightning as L
import numpy as np

import g_main
import module_model

shots = 1024

config = g_main.config
config["batch_size"]   = 10 # 300
config["rnd_seed"]     = 2
config["num_pt_ptcs"]  = 4
config["num_bin_data"] = 25 # 25 * 10 * 2 * (1-0.8) = 100 test data
L.seed_everything(config["rnd_seed"])

ibmq_dir = "./ibmq"
os.makedirs(ibmq_dir, exist_ok=True)

In [ ]:
class IBMQQCGNN(module_model.QCGNN):
    def forward(self, x):
        x = self.net(x)
        x = torch.unflatten(x, dim=-1, sizes=(2**self.num_ir_qubits, self.num_nr_qubits))
        x = x.mT
        meas = x.detach()
        x = torch.sum(x, dim=-1) * self.scale
        return x, meas

class QuantumRotQCGNN(nn.Module):
    def __init__(self, num_ir_qubits, num_nr_qubits, num_layers, num_reupload, quantum_config):
        super().__init__()
        if "qiskit" in quantum_config["qdevice"]:
            ctrl_enc = lambda _input, control_values: g_main.qiskit_encoding(_input, control_values, num_ir_qubits, num_nr_qubits)
        else:
            ctrl_enc = lambda _input, control_values: g_main.pennylane_encoding(_input, control_values, num_ir_qubits, num_nr_qubits)
        self.phi = IBMQQCGNN(num_ir_qubits, num_nr_qubits, num_layers, num_reupload, ctrl_enc=ctrl_enc, shots=shots, **quantum_config)
        self.mlp = module_model.ClassicalMLP(in_channel=num_nr_qubits, out_channel=1, hidden_channel=0, num_layers=0)
    
    def forward(self, x):
        x, meas = self.phi(x)
        x = self.mlp(x)
        return x.detach(), meas

In [ ]:
qidx = int(np.ceil(np.log2(config["num_pt_ptcs"])))
qnn, gl, gr  = 6, 1, 6
data_configs = [
    {"sig": "VzToZhToVevebb", "bkg": "VzToQCD", "abbrev":"BB-QCD", "cut": (800, 1000), "bin":10, "subjet_radius":0, "num_bin_data":config["num_bin_data"], "num_pt_ptcs":config["num_pt_ptcs"]},
    {"sig": "VzToTt", "bkg": "VzToQCD", "abbrev":"TT-QCD", "cut": (800, 1000), "bin":10, "subjet_radius":0, "num_bin_data":config["num_bin_data"], "num_pt_ptcs":config["num_pt_ptcs"]},
]

def load_state_dict(model, ckpt_path):
    old_state_dict = torch.load(ckpt_path)["state_dict"]
    new_state_dict = {}
    for old_key in old_state_dict.keys():
        new_key = old_key[6:]
        print(f"ModelLog: {old_key} ---> {new_key}")
        new_state_dict[new_key] = old_state_dict[old_key]
    model.load_state_dict(new_state_dict)

def prediction(x, quantum_config):
    # load model
    model_name   = QuantumRotQCGNN.__name__
    model_suffix = f"qidx3_qnn{qnn}_gl{gl}_gr{gr}"
    ckpt_key  = f"{model_name}_{model_suffix} | {data_config['abbrev']}_cut{data_config['cut']}"
    ckpt_path = g_main.get_ckpt_path(ckpt_key)
    model     = QuantumRotQCGNN(qidx, qnn, gl, gr, quantum_config)
    load_state_dict(model, ckpt_path)
    model.eval()
    return model(x)

ibmq_backend = input("Enter IBMQ backend = ")
for data_config in data_configs:
    data_module = g_main.generate_datamodule(data_config, graph=False)
    x, y_true, y_penl, meas_penl, y_ibmq, meas_ibmq = [torch.tensor([]) for _ in range(6)]
    for _x, _y_true in data_module.test_dataloader():
        _y_penl, _meas_penl = prediction(_x, {"qdevice": "default.qubit", "diff_method": "best", "qbackend": ""})
        _y_ibmq, _meas_ibmq = prediction(_x, {"qdevice": "qiskit.ibmq", "diff_method": "parameter-shift", "qbackend": ibmq_backend})
        x, y_true = torch.cat((x, _x)), torch.cat((y_true, _y_true))
        y_penl, meas_penl = torch.cat((y_penl, _y_penl)), torch.cat((meas_penl, _meas_penl))
        y_ibmq, meas_ibmq = torch.cat((y_ibmq, _y_ibmq)), torch.cat((meas_ibmq, _meas_ibmq))
    result = {"x":x, "y_true":y_true, "y_penl":y_penl, "meas_penl":meas_penl, "y_ibmq":y_ibmq, "meas_ibmq":meas_ibmq}
    npy_file = os.path.join(ibmq_dir, f"{ibmq_backend}-s{shots}-{data_config['abbrev']}_r{config['rnd_seed']}-p_{config['num_pt_ptcs']}_n{config['num_bin_data']}_b{config['batch_size']}-{qnn}_{gl}_{gr}.npy")
    np.save(npy_file, result, allow_pickle=True)